In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import argparse
import cv2
import imutils
import tensorflow as tf
from tensorflow import keras
from imutils.contours import sort_contours

In [2]:
from google.colab.patches import cv2_imshow

In [3]:
model=keras.models.load_model("./models/bestmodel1-1.hdf5")

In [4]:

class_mapping = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890'

In [ ]:

class_mapping = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890@#$%&(){}[],?'

In [5]:
labelNames = [l for l in class_mapping]

In [ ]:
#pre -proccessing image
count=0
avg_line_height=0
sum_line_height=0
image = cv2.imread('./images/phone_text4.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
# gray=cv2.resize(gray,(gray.shape[1]*8,gray.shape[0]*8), interpolation = cv2.INTER_LINEAR)
# cv2.imshow('gray',gray)
# cv2.waitKey(0)

image_h=gray.shape[0]
image_w=gray.shape[1]
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,81,15)
# cv2.imshow('second',thresh)
# cv2.waitKey(0)

kernel = np.ones((1,300), np.uint8)
img_dilation = cv2.dilate(thresh, kernel, iterations=1)
cv2_imshow(img_dilation)
# cv2.waitKey(0)

ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
width=0
height=0
# sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
sorted_ctrs = sort_contours(ctrs,"top-to-bottom")[0]
lines=[]
result=[]
results=np.array([[0,0,0,0]])
roi_number = 0
ROI_number = 0
roi2_number = 0
img1=255-thresh
for i, ctr in enumerate(sorted_ctrs):
    # Get bounding box
  x, y, w, h = cv2.boundingRect(ctr)
    # Getting ROI
  if h < np.round(image_h/100):
    continue
  sum_line_height+=h
  count+=1
avg_line_height=sum_line_height/count
print(avg_line_height)  

In [ ]:
##########################################
#predict 
######################################
lines=[]
result=[]
roi_number = 0
ROI_number = 0
roi2_number = 0
width=0
height=0
results=np.array([[0,0,0,0]])
image = cv2.imread('./images/phone_text4.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
# ratio=100/avg_line_height
# gray=cv2.resize(gray,(np.int(gray.shape[1]*ratio),np.int(gray.shape[0]*ratio)), interpolation = cv2.INTER_LINEAR)
image_h=gray.shape[0]
image_w=gray.shape[1]
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,81,12)
clone=np.copy(thresh)
cv2_imshow(thresh)
kernel = np.ones((1,100), np.uint8)
img_dilation = cv2.dilate(thresh, kernel, iterations=1)
cv2_imshow(img_dilation)
ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
sorted_ctrs = sort_contours(ctrs,"top-to-bottom")[0]
img1=255-thresh
for i, ctr in enumerate(sorted_ctrs):
  x, y, w, h = cv2.boundingRect(ctr)
  if h < np.round(image_h/100):
    continue
  roi = img1[y:y+h, x:x+w]
  cv2.imwrite('image_{}.png'.format(ROI_number), roi)
  roi_number += 1
  cv2.rectangle(clone,(x,y),( x + w, y + h ),(90,0,255),5)
  cv2.waitKey(0)
  cv2_imshow(roi)
  cv2.waitKey(0)
  lines.append(roi)
cv2_imshow(clone)
for line in lines:
  print(line.shape)
  line_h=line.shape[0]
  gray2=line
  thresh2 = cv2.adaptiveThreshold(gray2,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,81,8)
  cv2_imshow(thresh2)
  kernel2 = np.ones((1,20), np.uint8)
  img_dilation2 = cv2.dilate(thresh2, kernel2, iterations=1)
  ctrs2, hier2 = cv2.findContours(img_dilation2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  sorted_ctrs2 = sort_contours(ctrs2)[0]
  words_in_line=[]
  img2=255-thresh2
  roi2_number=0
  for i, ctr2 in enumerate(sorted_ctrs2):
    x, y, w, h = cv2.boundingRect(ctr2)
    if h<np.round(30):
      continue
    roi2 = img2[y:y+h, x:x+w]
    print(roi2.shape)
    roi2_number += 1
    cv2_imshow(roi2)
    words_in_line.append(roi2)

################################################
  print(len(words_in_line))
  for word in words_in_line:
    original_h=[]
    original_w=[]
    copyROIs=[]
    original = word.copy()
    gray3=word
    thresh3 = cv2.adaptiveThreshold(gray3,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,81,20)
    imgs=[]
    cnts3 = cv2.findContours(thresh3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts3 = cnts3[0] if len(cnts3) == 2 else cnts3[1]
    sorted_cnts3 = sort_contours(cnts3)[0]
    for c in sorted_cnts3:
      x,y,w,h = cv2.boundingRect(c)
      if h >= np.round(line_h/5) and h <= 120:
        cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
        ROI = original[y:y+h, x:x+w]
        original_h.append(ROI.shape[0])
        original_w.append(ROI.shape[1])
        cv2_imshow(ROI)
        ROI = cv2.resize(ROI, (60,60), interpolation = cv2.INTER_LINEAR)
        copyROIs.append(ROI.copy())
        ROI = np.expand_dims(ROI, 2)
        for i in range(ROI.shape[1]):
          for j in range(ROI.shape[0]):
            if ROI[i,j]<190 and ROI[i,j]>=170:
              ROI[i,j]= np.round(ROI[i,j]*0.3) 
            elif ROI[i,j]<170:
              ROI[i,j]=0
            elif ROI[i,j]<210 and ROI[i,j]>=190:
              ROI[i,j]= np.round(ROI[i,j]*0.6) 
        imgs.append(ROI)
        ROI_number += 1
    chars=(255-np.array(imgs))/255
    
    if(len(chars)>=1):
      preds=np.argmax(model.predict(chars),axis=1)
      probs=np.max(model.predict(chars),axis=1)
      w=""
      k=0
      for pred in preds:
        label = labelNames[pred]
        w+=label
        results=np.append(results,[[label,original_h[k],original_w[k],copyROIs[k]]],axis=0)
        height+=original_h[k]
        width+=original_w[k]
        k+=1
      result.append(w)
      results=np.append(results,[[" "," "," "," "]],axis=0)
print(" ".join(result))
print("#################")
print(height/ROI_number)
print(width/ROI_number)
results=np.delete(results,0,0)

In [ ]:
results[:,0]

In [ ]:
il=np.mean((np.array(results[:,1])[[i for i,v in enumerate(results[:,0]) if v=='i' or v=='l']]).astype(np.float))

In [ ]:
il

67.82

In [ ]:
resultss=""

In [ ]:
numbers=['1','2','3','4','5','6','7','8','9']
wide_chars=['C',  'T', 'W', 'w','P','A']

In [ ]:
for i in range(results.shape[0]):
  print(i)
  if type(results[i,2])!=str:
    cv2_imshow(results[i,3])
  if results[i,0]=='i' or results[i,0]=='l' or results[i,0]=='I':
    if results[i,1]<=il:
      results[i,0]='i'
    elif results[i,1]>il:
      results[i,0]='l'
  if results[i,0]=='m':
    resultss+=results[i,0]
    continue
  # if type(results[i,2])!=str and results[i,2]>=(width/ROI_number*1.3)and results[i,0] not in wide_chars:
  #   images=[]
  #   image = results[i,3]
  #   image_w=image.shape[1]
  #   print(image.shape)
  #   a=np.abs(np.sum(image,axis=0).argsort()[-10:][::-1])
  #   index=np.int(image_w/2)
  #   # print(index)
  #   # print(a)
  #   for j in a:
  #     if j>60/5 and j<60*0.8:
  #       index=j
  #       break
  #   image1=image[:,:index]
  #   cv2_imshow(image)
  #   print(image1.shape)
  #   cv2_imshow(image1)
  #   image1 = cv2.resize(image1, (60,60), interpolation = cv2.INTER_LINEAR)
  #   images.append(image1)
  #   image2=image[:,index:]
  #   print(image2.shape)
  #   cv2_imshow(image2)
  #   image2 = cv2.resize(image2, (60,60), interpolation = cv2.INTER_LINEAR)
  #   images.append(image2)
  #   chars=255-np.array(images)
  #   preds=np.argmax(model.predict(chars),axis=1)
  #   chars=''
  #   for pred in preds:
  #     label = labelNames[pred]
  #     print(label)
  #     chars+=label
  #   results[i,0]=chars
  #   print(chars)
  # if results[i,0]=='in' or results[i,0]=='In' or results[i,0]=='ln' or results[i,0]=='1n' or results[i,0]=='rn' :
  #   results[i,0]='m'
  if results[i,0]=='.':
    if results[i-1,0]=='i' or results[i+1,0]=='i' or results[i+1,0]=='l':
      results[i,0]=''
  if results[i,0]=='0' and (results[i-1,0] not in numbers or results[i+1,0] not in numbers):
    results[i,0]='o'
  if results[i,0]=='1' and (results[i-1,0] not in numbers or results[i+1,0] not in numbers):
    results[i,0]='i'
  if results[i,0]=='5' and (results[i-1,0] not in numbers or results[i+1,0] not in numbers):
    results[i,0]='s'
  print(results[i,0])
  resultss+=results[i,0]
print(resultss.lower())

In [ ]:
##########################################
#cut into lines
######################################
image = cv2.imread('./images/phone_text7.jpg')

gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

cv2_imshow(gray)
print(gray.shape)


thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,121,35)
cv2_imshow(thresh)

kernel = np.ones((1,300), np.uint8)
img_dilation = cv2.dilate(thresh, kernel, iterations=1)
cv2_imshow(img_dilation)


ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
width=0
height=0

sorted_ctrs = sort_contours(ctrs,"top-to-bottom")[0]
lines=[]
result=[]
roi_number = 0
ROI_number = 0
roi2_number = 0
img1=255-thresh
for i, ctr in enumerate(sorted_ctrs):
    # Get bounding box
  x, y, w, h = cv2.boundingRect(ctr)

  if h < 10:
    continue
  roiroi=gray[y:y+h, x:x+w]
  cv2_imshow(roiroi)
  roi = img1[y:y+h, x:x+w]
    # cv2.imwrite('image_{}.png'.format(ROI_number), roi)
  roi_number += 1
    # show ROI
  print(roi_number)
  print(roi.shape)
  cv2_imshow(roi)
  # cv2.rectangle(image,(x,y),( x + w, y + h ),(90,0,255),2)
  cv2.waitKey(0)
  cv2.imwrite('line_{}.png'.format(roi_number), roi)
  print("++++++++++")
  # print(roi.shape)
  # roi=cv2.resize(roi,(roi.shape[1]*2,roi.shape[0]*2), interpolation = cv2.INTER_AREA)
  lines.append(roi)
# cv2_imshow(image)
print(len(lines))

In [ ]:
# cut into words
#change the path
line = cv2.imread('/content/line_2.png')
print(line.shape)

gray2 = cv2.cvtColor(line,cv2.COLOR_BGR2GRAY)
gray2=cv2.resize(gray2,(gray2.shape[1]*2,gray2.shape[0]*2), interpolation = cv2.INTER_LINEAR)
cv2_imshow(gray2)
print(gray2.shape)
# cv2.waitKey(0)

thresh2 = cv2.adaptiveThreshold(gray2,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,81,8)
cv2_imshow(thresh2)
# cv2.waitKey(0)

kernel2 = np.ones((5,25), np.uint8)
img_dilation2 = cv2.dilate(thresh2, kernel2, iterations=1)
cv2_imshow(img_dilation2)
  # cv2.waitKey(0)

ctrs2, hier2 = cv2.findContours(img_dilation2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
sorted_ctrs2 = sort_contours(ctrs2)[0]
words_in_line=[]
img2=255-thresh2
roi2_number=0
for i, ctr2 in enumerate(sorted_ctrs2):
    # Get bounding box
  x, y, w, h = cv2.boundingRect(ctr2)
    # Getting ROI
  if h<10:
    continue
  roi2 = img2[y:y+h, x:x+w]
  print(roi2_number)
  print(roi2.shape)
  cv2.imwrite('word_{}.png'.format(roi2_number), roi2)
  roi2_number += 1
    # show ROI
  cv2_imshow(roi2)
  cv2.rectangle(thresh2,(x,y),( x + w, y + h ),(90,0,255),5)
    # cv2.waitKey(0)
    
  words_in_line.append(roi2)
  cv2_imshow(thresh2)

In [ ]:
##################################################
# cut into characters
###################################################
result=[]
#change the path
image = cv2.imread("/content/word_1.png")
original = image.copy()
# image=cv2.resize(image,(image.shape[1]*2,image.shape[0]*2))
gray3 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# thresh3 = cv2.threshold(gray3, 120, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
thresh3 = cv2.adaptiveThreshold(gray3,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,81,12)
cv2_imshow(gray3)
# Find contours, obtain bounding box, extract and save ROI
imgs=[]
roi3_number = 0

cnts3 = cv2.findContours(thresh3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts3 = cnts3[0] if len(cnts3) == 2 else cnts3[1]
sorted_cnts3 = sort_contours(cnts3)[0]
for c in sorted_cnts3:
  x,y,w,h = cv2.boundingRect(c)
  if  h >= 1 and h <= 500:
    # cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
    # if x>4:
    #   x-=4
    # w+=8
    # if y>10:
    #   y-=10
    # h+=20
    ROI = original[y:y+h, x:x+w]
    # expand_h=int((28-ROI.shape[0])/2)
    # expand_r=int((28-ROI.shape[1])/2)
    # if expand_h>0 and expand_r>0:
    #   ROI=cv2.copyMakeBorder(ROI,expand_h,28-(expand_h+ROI.shape[0]),expand_r,28-(expand_r+ROI.shape[1]),borderType=cv2.BORDER_CONSTANT,value=[255,255,255])
    # else:
    print(roi3_number)
    print(ROI.shape)
    cv2_imshow(ROI)
    ROI = cv2.resize(ROI, (60,60), interpolation = cv2.INTER_LINEAR)
      
    ROI= ROI[:,:,0]
    ROI = np.expand_dims(ROI, 2)
    for i in range(ROI.shape[1]):
      for j in range(ROI.shape[0]):
        if ROI[i,j]<190 and ROI[i,j]>=170:
         ROI[i,j]= np.round(ROI[i,j]*0.3) 
        elif ROI[i,j]<170:
          ROI[i,j]=0
        elif ROI[i,j]<210 and ROI[i,j]>=190:
          ROI[i,j]= np.round(ROI[i,j]*0.6) 
    # cv2_imshow(ROI)
    cv2.rectangle(image,(x,y),( x + w, y + h ),(1,0,255),1)
    cv2.waitKey(0)
    cv2.imwrite('char_{}.png'.format(roi3_number), ROI)
    imgs.append(ROI)
    roi3_number += 1
cv2_imshow(image)
chars=255-np.array(imgs)
if(len(chars)>=1):
  # print(model.predict(chars))
  preds=np.argmax(model.predict(chars),axis=1)
  w=""
  for pred in preds:
	  label = labelNames[pred]
	  w+=label
  result.append(w)
print(" ".join(result))
